In [56]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.stats import t

def process_file(file_path):
    font_legend = {'family': 'TH Sarabun New',
                   'weight': 'bold',
                   'style': 'normal',
                   'size': 18}
    font_axis = {'family': 'TH Sarabun New',
                 'weight': 'bold',
                 'style': 'normal',
                 'size': 20}

    base_name  = os.path.splitext(os.path.basename(file_path))[0]
    parent_dir = os.path.dirname(file_path)
    output_dir = os.path.join(parent_dir, f"{base_name} plots")
    os.makedirs(output_dir, exist_ok=True)

    # Read CSV, skip first two summary rows, use third row as header
    df = pd.read_csv(file_path, delimiter=',', skiprows=2, header=0)
    x  = df['Run Time [s]']

    # 1) Mass Flow Rate vs Run Time (full) — raw data only, no curve fit
    plt.figure()
    plt.plot(x, df['Weight [Kg]'], label='Data')
    plt.xlabel('Run Time [s]', fontdict=font_axis)
    plt.ylabel('Weight [Kg]', fontdict=font_axis)
    plt.title('Mass Flow Rate vs Run Time', fontdict=font_axis)
    plt.legend(prop=font_legend)
    plt.ylim(bottom=0)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_Mass_Flow_Rate_full_plot.png"))
    plt.close()

    # ... (Differential Pressure, Rotational Speed, Torque full-plot blocks unchanged) ...

    # --- FILTERED PLOTS (20 ≤ Weight ≤ 30 Kg) ---
    filtered = df[(df['Weight [Kg]'] >= 20) & (df['Weight [Kg]'] <= 30)]
    if not filtered.empty:
        
        frt = filtered['Run Time [s]'].iloc[-1] - filtered['Run Time [s]'].iloc[0]

        # Mass Flow Rate (filtered fit) + capture its standard error
        xf, yf = filtered['Run Time [s]'], filtered['Weight [Kg]']
        slope_f, intercept_f, r_f, _, stderr_f = linregress(xf, yf)
        # convert 1σ stderr into 95% CI half-width
        dof    = len(xf) - 2
        t_crit = t.ppf(0.975, dof)
        stderr_f *= t_crit
        r2_f = r_f**2
        max_mass = max(yf)
        min_mass = min(yf)
        plt.figure()
        plt.plot(xf, yf, label='Filtered')
        plt.plot(xf, slope_f*xf + intercept_f,
                 label=f'Fit: y={slope_f:.4f}x+{intercept_f:.4f}\nR²={r2_f:.4f}')
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Weight [Kg]', fontdict=font_axis)
        plt.title('Mass Flow Rate vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95, f'Filtered Run Time: {frt:.2f} s',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.legend(prop=font_legend); plt.ylim(bottom=0)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Mass_Flow_Rate_filtered_plot.png"))
        plt.close()

        # Differential Pressure stats + plot
        dp     = filtered['Differential Pressure [Bar]']
        avg_dp = dp.mean()
        std_dp = 2 * dp.std(ddof=1)
        plt.figure()
        plt.plot(xf, dp)
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Differential Pressure [Bar]', fontdict=font_axis)
        plt.title('Differential Pressure vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s\n'
                 f'Avg: {avg_dp:.4f} Bar\n'
                 f'SD:  {std_dp:.4f}',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.ylim([0,3])
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Differential_Pressure_filtered_plot.png"))
        plt.close()

        # Rotational Speed stats + plot
        rs     = filtered['Rotational Speed [RPM]']
        avg_rs = rs.mean()
        std_rs = 2 * rs.std(ddof=1)
        plt.figure()
        plt.plot(xf, rs)
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Rotational Speed [RPM]', fontdict=font_axis)
        plt.title('Rotational Speed vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s\n'
                 f'Avg: {avg_rs:.4f} RPM\n'
                 f'SD:  {std_rs:.4f}',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.ylim([0,8500])
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Rotational_Speed_filtered_plot.png"))
        plt.close()

        # Torque stats + plot
        tq     = filtered['Torque [N-m]']
        avg_tq = tq.mean()
        std_tq = 2 * tq.std(ddof=1)
        plt.figure()
        plt.plot(xf, tq)
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Torque [N-m]', fontdict=font_axis)
        plt.title('Torque vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s\n'
                 f'Avg: {avg_tq:.4f} N-m\n'
                 f'SD:  {std_tq:.4f}',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.ylim([0,0.5])
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Torque_filtered_plot.png"))
        plt.close()
        
    else:
        slope_f = stderr_f = np.nan
        avg_dp = avg_rs = avg_tq = std_dp = std_rs = std_tq = np.nan
        print(f"{base_name}: no data in weight range 20–30 Kg")

    # ---- RETURN original metrics PLUS S.E. of slope ----
    return {
        'file':                base_name,
        'mass_flow_rate':      slope_f,
        'se_mass_flow_rate':   stderr_f,     # added standard error
        'mean_dp':             avg_dp,
        'mean_rs':             avg_rs,
        'mean_torque':         avg_tq,
        'std_dp':              std_dp,
        'std_rs':              std_rs,
        'std_tq':              std_tq,
        'run_time':            frt,
        'max_mass':            max_mass,
        'min_mass':            min_mass
    }


def main(csv_dir):
    files = glob.glob(os.path.join(csv_dir, '*.csv'))
    if not files:
        print(f"No CSV files found in {csv_dir}")
        return

    summaries = [process_file(f) for f in files]

    # ARRAYS of the PER-FILE time-averaged MEANS (unchanged)
    mfr_arr = np.array([s['mass_flow_rate'] for s in summaries])
    dp_arr  = np.array([s['mean_dp']        for s in summaries])
    rs_arr  = np.array([s['mean_rs']        for s in summaries])
    tq_arr  = np.array([s['mean_torque']    for s in summaries])
    run_time_arr = np.array([s['run_time']    for s in summaries])
    max_mass_arr = np.array([s['max_mass']    for s in summaries])
    min_mass_arr = np.array([s['min_mass']    for s in summaries])
    
    # ARRAYS of the PER-FILE SDs (unchanged)
    sd_dp_arr = np.array([s['std_dp'] for s in summaries])
    sd_rs_arr = np.array([s['std_rs'] for s in summaries])
    sd_tq_arr = np.array([s['std_tq'] for s in summaries])

    # ARRAY of the PER-FILE S.E.
    se_arr = np.array([s['se_mass_flow_rate'] for s in summaries])

    # GRAND MEANS & SDs of the MEANS (unchanged)
    grand_means = {
        'mfr_mean':  np.nanmean(mfr_arr), 'mfr_sd':  np.nanstd(mfr_arr, ddof=1),
        'dp_mean':   np.nanmean(dp_arr),  'dp_sd':   np.nanstd(dp_arr,  ddof=1),
        'rs_mean':   np.nanmean(rs_arr),  'rs_sd':   np.nanstd(rs_arr,  ddof=1),
        'tq_mean':   np.nanmean(tq_arr),  'tq_sd':   np.nanstd(tq_arr,  ddof=1),
        'run_time_mean':   np.nanmean(run_time_arr),  'run_time_sd':   np.nanstd(run_time_arr,  ddof=1),
        'max_mass_mean':   np.nanmean(max_mass_arr),  'max_mass_sd':   np.nanstd(max_mass_arr,  ddof=1),
        'min_mass_mean':   np.nanmean(min_mass_arr),  'min_mass_sd':   np.nanstd(min_mass_arr,  ddof=1),
    } #Single Number

    # ARITHMETIC AVERAGE OF THE SDs (unchanged)
    avg_sd = {
        'dp_bar': np.nanmean(sd_dp_arr),
        'rs_bar': np.nanmean(sd_rs_arr),
        'tq_bar': np.nanmean(sd_tq_arr),
    }

    # AVERAGE STANDARD ERROR OF MASS FLOW RATE (added)
    avg_se = np.nanmean(se_arr)

    #Precision Uncertainty Calculation Initialization
    Ns = len(dp_arr)
    dof = Ns - 1  # degrees of freedom for mean
    t_crit = t.ppf(0.975, dof) #0.975 for two-tails student-t
    
    grand_pu = {
        'mfr_pu':  t_crit * np.nanstd(mfr_arr, ddof=1) / np.sqrt(Ns),
        'dp_pu':   t_crit * np.nanstd(dp_arr, ddof=1) / np.sqrt(Ns),
        'rs_pu':   t_crit * np.nanstd(rs_arr, ddof=1) / np.sqrt(Ns),
        'tq_pu':   t_crit * np.nanstd(tq_arr, ddof=1) / np.sqrt(Ns)
    } #Single Number
    
    #Bias Uncertainty Code Area
    Accuracy_Dict = {
        'mass_acc': 0.01, 'mass_res': 10 / 2**16 * 5, #kg
        'time_acc': 50e-6 * 0.05, 'time_res': 50e-9, #sec
        'dp_acc': 0.0075, 'dp_res': 10 / 2**16 * 0.3, #bar, res = 10V / 16-bit * 0.3 Bar/V, acc = 0.25%FS
        'tq_acc': 0.01, 'tq_res': 10 / 2**16 * 0.1, #N-m, res = 10V / 16-bit * 0.1 N-m/V
        'rs_acc': 0, 'rs_res': 0 #RPM   
    } #Single Number
    
    mass_bu = np.sqrt((Accuracy_Dict['mass_acc'])**2 + (Accuracy_Dict['mass_res'] / 2)**2)
    time_bu = np.sqrt((Accuracy_Dict['time_acc'])**2 + (Accuracy_Dict['time_res'] / 2)**2)
    dp_bu = np.sqrt((Accuracy_Dict['dp_acc'])**2 + (Accuracy_Dict['dp_res'] / 2)**2)
    rs_bu = np.sqrt((Accuracy_Dict['rs_acc'])**2 + (Accuracy_Dict['rs_res'] / 2)**2)
    tq_bu = np.sqrt((Accuracy_Dict['tq_acc'])**2 + (Accuracy_Dict['tq_res'] / 2)**2)
    mfr_bu = np.sqrt((mass_bu/grand_means['run_time_mean'])**2 + ((grand_means['max_mass_mean'] - grand_means['min_mass_mean'])/grand_means['run_time_mean']**2*time_bu)**2)
    
    #Final Sum Uncertainty
    mfr_uncertainty = np.sqrt((mfr_bu)**2 + (grand_pu['mfr_pu'])**2 + (avg_se)**2)
    dp_uncertainty = np.sqrt((dp_bu)**2 + (dp_bu)**2 + (avg_sd['dp_bar'])**2)
    rs_uncertainty = np.sqrt((rs_bu)**2 + (rs_bu)**2 + (avg_sd['rs_bar'])**2)
    tq_uncertainty = np.sqrt((tq_bu)**2 + (tq_bu)**2 + (avg_sd['tq_bar'])**2)
    
    # --- PRINT ---
    print("\nPer-file filtered MEANS:")
    for s in summaries:
        print(f" - {s['file']}: Mass Flow Rate={s['mass_flow_rate']:.4f}, "
              f"ΔPs={s['mean_dp']:.4f}, RPM={s['mean_rs']:.2f}, "
              f"Torque={s['mean_torque']:.4f}")

    print("\nGrand trimmed summary (of MEANS):")
    print(f" • Mass Flow Rate: mean={grand_means['mfr_mean']:.4f}, SD={grand_means['mfr_sd']:.4f}, Precision Uncertainty={grand_pu['mfr_pu']:.4f}, Bias Uncertainty={mfr_bu:.4f}")
    print(f" • ΔPs:             mean={grand_means['dp_mean']:.4f}, SD={grand_means['dp_sd']:.4f}, Precision Uncertainty={grand_pu['dp_pu']:.4f}, Bias Uncertainty={dp_bu:.4f}")
    print(f" • RPM:            mean={grand_means['rs_mean']:.2f}, SD={grand_means['rs_sd']:.2f}, Precision Uncertainty={grand_pu['rs_pu']:.4f}, Bias Uncertainty={rs_bu:.4f}")
    print(f" • Torque:         mean={grand_means['tq_mean']:.4f}, SD={grand_means['tq_sd']:.4f}, Precision Uncertainty={grand_pu['tq_pu']:.4f}, Bias Uncertainty={tq_bu:.4f}")


    print("\nPer-file filtered SDs:")
    for s in summaries:
        print(
            f" - {s['file']}: "
            f"SE Mass Flow Rate={s['se_mass_flow_rate']:.4f}, "
            f"2*SD ΔPs={s['std_dp']:.4f}, "
            f"2*SD RPM={s['std_rs']:.4f}, "
            f"2*SD Torque={s['std_tq']:.4f}"
        )

    print("\nAverage of SDs (SD̄):")
    print(f" • ΔPs 2*SD̄ = {avg_sd['dp_bar']:.4f}")
    print(f" • RPM 2*SD̄= {avg_sd['rs_bar']:.4f}")
    print(f" • Torque 2*SD̄ = {avg_sd['tq_bar']:.4f}")

    # Print average standard error
    print(f"\nAverage standard error of Mass Flow Rate = {avg_se:.4f}\n")

    # --- WRITE summary.txt ---
    folder = os.path.basename(csv_dir.rstrip(os.sep))
    summary_path = os.path.join(csv_dir, f"{folder} summary.txt")
    with open(summary_path, 'w', encoding='utf-8') as f:
        
        f.write("\nTime-Averaged Values:\n")
        for s in summaries:
            f.write(f"{s['file']}: Mass Flow Rate={s['mass_flow_rate']:.4f}, "
                    f"ΔPs={s['mean_dp']:.4f}, RPM={s['mean_rs']:.2f}, "
                    f"Torque={s['mean_torque']:.4f}\n")
            
        f.write("\nTime-Averaged S.D. / S.E.:\n")
        
        for s in summaries:
            f.write(
                f"{s['file']}: "
                f"SE (95% CI) Mass Flow Rate={s['se_mass_flow_rate']:.4f}, "
                f"2*SD ΔPs={s['std_dp']:.4f}, "
                f"2*SD RPM={s['std_rs']:.4f}, "
                f"2*SD Torque={s['std_tq']:.4f}\n"
            )
        
        f.write("\nInstance Average of Means Value:\n")
        f.write(f"Mass Flow Rate: mean={grand_means['mfr_mean']:.4f}, SD={grand_means['mfr_sd']:.4f}, Precision Uncertainty={grand_pu['mfr_pu']:.4f}, Bias Uncertainty={mfr_bu:.4f}\n")
        f.write(f"ΔPs:             mean={grand_means['dp_mean']:.4f}, SD={grand_means['dp_sd']:.4f}, Precision Uncertainty={grand_pu['dp_pu']:.4f}, Bias Uncertainty={dp_bu:.4f}\n")
        f.write(f"RPM:            mean={grand_means['rs_mean']:.2f}, SD={grand_means['rs_sd']:.2f}, Precision Uncertainty={grand_pu['rs_pu']:.4f}, Bias Uncertainty={rs_bu:.4f}\n")
        f.write(f"Torque:         mean={grand_means['tq_mean']:.4f}, SD={grand_means['tq_sd']:.4f}, Precision Uncertainty={grand_pu['tq_pu']:.4f}, Bias Uncertainty={tq_bu:.4f}\n")
        
        f.write("\nInstance Average of Time-Averaged SDs (SD̄):\n")
        f.write(f"S.E. (95% CI) of Mass Flow Rate = {avg_se:.4f}\n")
        f.write(f"ΔPs 2*SD̄ = {avg_sd['dp_bar']:.4f}\n")
        f.write(f"RPM 2*SD̄= {avg_sd['rs_bar']:.4f}\n")
        f.write(f"Torque 2*SD̄ = {avg_sd['tq_bar']:.4f}\n")

        f.write("\nFinal Uncertainty:\n")
        f.write(f"Uncertainty of Mass Flow Rate = {mfr_uncertainty:.4f} Kg/s\n")
        f.write(f"Uncertainty of ΔPs (RAW) = {dp_uncertainty:.4f} Bar\n")
        f.write(f"Uncertainty of RPM = {rs_uncertainty:.4f} RPM\n")
        f.write(f"Uncertainty of Torque = {tq_uncertainty:.4f} N-m\n")


    print(f"Summary written to {summary_path}")

if __name__ == '__main__':
    # 1) Base path (up to “Glycerine\”)
    BASE_DIR = r'C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine'
    
    # 2) List the final two-level folders you want to process:
    subdirs = [
        r'24042025\Glycerine_8%_6730_0.081Kg-s',
        r'24042025\Glycerine_8%_7760_0.094Kg-s',
        r'23042025\Glycerine_8%_7250_0.094Kg-s',
        r'23042025\Glycerine_85%_6730_0.526Kg-s',
        r'23042025\Glycerine_85%_7760_0.603Kg-s',
        r'22042025\Glycerine_52.5%_6730_0.454Kg-s',
        r'22042025\Glycerine_52.5%_7760_0.521Kg-s',
        r'22042025\Glycerine_85%_7250_0.565Kg-s',
        r'21042025\Glycerine_43%_6730_0.416Kg-s',
        r'21042025\Glycerine_43%_7250_0.447Kg-s',
        r'21042025\Glycerine_43%_7760_0.477Kg-s',
        r'21042025\Glycerine_52.5%_7250_0.487Kg-s',
        r'20042025\Glycerine_31.5%_6730_0.344Kg-s',
        r'20042025\Glycerine_31.5%_7250_0.370Kg-s',
        r'20042025\Glycerine_36.5%_6730_0.378Kg-s',
        r'20042025\Glycerine_36.5%_7250_0.408Kg-s',
        r'19042025\Glycerine_13%_6730_0.160Kg-s',
        r'19042025\Glycerine_13%_7760_0.188Kg-s',
        r'18042025\Glycerine_13%_7250_0.175Kg-s',
        r'18042025\Glycerine_19%_7760_0.268Kg-s',
        r'17042025\Glycerine_19%_6730_0.235Kg-s',
        r'17042025\Glycerine_19%_7250_0.253Kg-s',
        r'17042025\Glycerine_25.6%_6730_0.302Kg-s',
        r'17042025\Glycerine_25.6%_7250_0.328Kg-s',
        r'16042025\Glycerine_21%_6730_0.272Kg-s',
        r'26042025\Glycerine_27.5%_7760_0.354Kg-s',
        r'26042025\Glycerine_21%_7760_0.313Kg-s',
        r'26042025\Glycerine_21%_7250_0.281Kg-s',
        r'28042025\Glycerine_13%_6730_0.160Kg-s',
        r'28042025\Glycerine_21%_7250_0.290Kg-s',
        r'28042025\Glycerine_31.5%_7760_0.395Kg-s',
        r'28042025\Glycerine_36.5_7760_0.437Kg-s',
        r'28042025\Glycerine_52.5%_7760_0.521Kg-s',
        r'29042025\Glycerine_13%_7760_0.187Kg-s',
    ]

    # 3) Iterate through each and run
    for sub in subdirs:
        csv_directory = os.path.join(BASE_DIR, sub)
        print(f"\n=== Processing folder: {csv_directory} ===")
        main(csv_directory)



=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\24042025\Glycerine_8%_6730_0.081Kg-s ===

Per-file filtered MEANS:
 - Glycerine_24042025_1354_6730RPM_8%_#1_0.081Kg-s: Mass Flow Rate=0.0797, ΔPs=1.9931, RPM=6729.90, Torque=0.3364
 - Glycerine_24042025_1412_6730RPM_8%_#2_0.081Kg-s: Mass Flow Rate=0.0802, ΔPs=1.9987, RPM=6730.00, Torque=0.3304
 - Glycerine_24042025_1430_6730RPM_8%_#3_0.080Kg-s: Mass Flow Rate=0.0798, ΔPs=1.9987, RPM=6730.06, Torque=0.3272
 - Glycerine_24042025_1448_6730RPM_8%_#4_0.080Kg-s: Mass Flow Rate=0.0796, ΔPs=2.0049, RPM=6729.99, Torque=0.3152
 - Glycerine_24042025_1506_6730RPM_8%_#5_0.079Kg-s: Mass Flow Rate=0.0787, ΔPs=2.0072, RPM=6729.99, Torque=0.2865

Grand trimmed summary (of MEANS):
 • Mass Flow Rate: mean=0.0796, SD=0.0005, Precision Uncertainty=0.0007, Bias Uncertainty=0.0001
 • ΔPs:             mean=2.0005, SD=0.0056, Precision Uncertainty=0.0070, Bias Uncertainty=0.0075


In [4]:
 subdirs = [
        r'24042025/Glycerine_8%_6730_0.081Kg-s',
        r'24042025/Glycerine_8%_7760_0.094Kg-s',
        r'23042025/Glycerine_8%_7250_0.094Kg-s',
        r'23042025/Glycerine_85%_6730_0.526Kg-s',
        r'23042025/Glycerine_85%_7760_0.603Kg-s',
        r'22042025/Glycerine_52.5%_6730_0.454Kg-s',
        r'22042025/Glycerine_52.5%_7760_0.521Kg-s',
        r'22042025/Glycerine_85%_7250_0.565Kg-s',
        r'21042025/Glycerine_43%_6730_0.416Kg-s',
        r'21042025/Glycerine_43%_7250_0.447Kg-s',
        r'21042025/Glycerine_43%_7760_0.477Kg-s',
        r'21042025/Glycerine_52.5%_7250_0.487Kg-s',
        r'20042025/Glycerine_31.5%_6730_0.344Kg-s',
        r'20042025/Glycerine_31.5%_7250_0.370Kg-s',
        r'20042025/Glycerine_36.5%_6730_0.378Kg-s',
        r'20042025/Glycerine_36.5%_7250_0.408Kg-s',
        r'19042025/Glycerine_13%_6730_0.160Kg-s',
        r'19042025/Glycerine_13%_7760_0.188Kg-s',
        r'18042025/Glycerine_13%_7250_0.175Kg-s',
        r'18042025/Glycerine_19%_7760_0.268Kg-s',
        r'17042025/Glycerine_19%_6730_0.235Kg-s',
        r'17042025/Glycerine_19%_7250_0.253Kg-s',
        r'17042025/Glycerine_25.6%_6730_0.302Kg-s',
        r'17042025/Glycerine_25.6%_7250_0.328Kg-s',
        r'16042025/Glycerine_21%_6730_0.272Kg-s',
        r'26042025/Glycerine_27.5%_7760_0.354Kg-s',
        r'26042025/Glycerine_21%_7760_0.313Kg-s',
        r'26042025/Glycerine_21%_7250_0.281Kg-s',
        r'28042025/Glycerine_13%_6730_0.160Kg-s',
        r'28042025/Glycerine_21%_7250_0.290Kg-s',
        r'28042025/Glycerine_31.5%_7760_0.395Kg-s',
        r'28042025/Glycerine_36.5_7760_0.437Kg-s',
        r'28042025/Glycerine_52.5%_7760_0.521Kg-s',
        r'29042025/Glycerine_13%_7760_0.187Kg-s',
    ]
 
   BASE_DIR = r'/Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine'
 
 
 
 #Windows
 
  subdirs = [
        r'24042025\Glycerine_8%_6730_0.081Kg-s',
        r'24042025\Glycerine_8%_7760_0.094Kg-s',
        r'23042025\Glycerine_8%_7250_0.094Kg-s',
        r'23042025\Glycerine_85%_6730_0.526Kg-s',
        r'23042025\Glycerine_85%_7760_0.603Kg-s',
        r'22042025\Glycerine_52.5%_6730_0.454Kg-s',
        r'22042025\Glycerine_52.5%_7760_0.521Kg-s',
        r'22042025\Glycerine_85%_7250_0.565Kg-s',
        r'21042025\Glycerine_43%_6730_0.416Kg-s',
        r'21042025\Glycerine_43%_7250_0.447Kg-s',
        r'21042025\Glycerine_43%_7760_0.477Kg-s',
        r'21042025\Glycerine_52.5%_7250_0.487Kg-s',
        r'20042025\Glycerine_31.5%_6730_0.344Kg-s',
        r'20042025\Glycerine_31.5%_7250_0.370Kg-s',
        r'20042025\Glycerine_36.5%_6730_0.378Kg-s',
        r'20042025\Glycerine_36.5%_7250_0.408Kg-s',
        r'19042025\Glycerine_13%_6730_0.160Kg-s',
        r'19042025\Glycerine_13%_7760_0.188Kg-s',
        r'18042025\Glycerine_13%_7250_0.175Kg-s',
        r'18042025\Glycerine_19%_7760_0.268Kg-s',
        r'17042025\Glycerine_19%_6730_0.235Kg-s',
        r'17042025\Glycerine_19%_7250_0.253Kg-s',
        r'17042025\Glycerine_25.6%_6730_0.302Kg-s',
        r'17042025\Glycerine_25.6%_7250_0.328Kg-s',
        r'16042025\Glycerine_21%_6730_0.272Kg-s',
        r'26042025\Glycerine_27.5%_7760_0.354Kg-s',
        r'26042025\Glycerine_21%_7760_0.313Kg-s',
        r'26042025\Glycerine_21%_7250_0.281Kg-s',
        r'28042025\Glycerine_13%_6730_0.160Kg-s',
        r'28042025\Glycerine_21%_7250_0.290Kg-s',
        r'28042025\Glycerine_31.5%_7760_0.395Kg-s',
        r'28042025\Glycerine_36.5_7760_0.437Kg-s',
        r'28042025\Glycerine_52.5%_7760_0.521Kg-s',
        r'29042025\Glycerine_13%_7760_0.187Kg-s',
    ]

  BASE_DIR = r'C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine'

IndentationError: unindent does not match any outer indentation level (<string>, line 44)

In [3]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import matplotlib.font_manager as font_manager

def process_file(file_path):
    font_legend = {'family': 'TH Sarabun New',
                    'weight': 'bold',
                    'style': 'normal', 
                    'size': 18
                   }
    font_axis = {'family': 'TH Sarabun New',
                    'weight': 'bold',
                    'style': 'normal', 
                    'size': 20
                   }
    base_name   = os.path.splitext(os.path.basename(file_path))[0]
    parent_dir  = os.path.dirname(file_path)
    output_dir  = os.path.join(parent_dir, f"{base_name} plots")
    output_dir  = os.path.join(output_dir, f"combined plots")
    os.makedirs(output_dir, exist_ok=True)

    # Read CSV, skip first two summary rows, use third row as header
    df = pd.read_csv(file_path, delimiter=',', skiprows=2, header=0)

    # --- COMBINED FULL PLOTS ---

    # 1) Torque and Pressure vs Weight
    x = df['Weight [Kg]']
    #y = df['Weight [Kg]']

    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    #plt.plot(x, y, label='Data')
    #plt.xlabel('Run Time [s]')
    #plt.ylabel('Weight [Kg]')
    #plt.title('Mass Flow Rate vs Run Time')
    #plt.legend()
    #plt.ylim(bottom=0)
    #plt.tight_layout()

    lns1 = ax1.plot(x, df['Differential Pressure [Bar]'],label='Differential Pressure')
    ax1.set_xlabel('Weight [Kg]', fontdict=font_axis)
    ax1.set_ylabel('Differential Pressure [Bar]', fontdict=font_axis)
    ax1.set_ylim(bottom=0)

    lns2 = ax2.plot(x, df["Torque [N-m]"],color="orange",label='Torque')
    ax2.set_ylabel('Torque [N-m]', fontdict=font_axis)
    ax2.set_ylim([0,0.6])
    
    lns = lns1 + lns2
    
    labs = [l.get_label() for l in lns]       
    ax1.legend(lns, labs, loc="lower right",prop=font_legend)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_Pressure_&_Torque_vs_Weight_full_combined_plot.png"))
    plt.close()
    
    # 2) Torque and Pressure vs Time
    x = df['Run Time [s]']
    #y = df['Weight [Kg]']

    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    #plt.plot(x, y, label='Data')
    #plt.xlabel('Run Time [s]')
    #plt.ylabel('Weight [Kg]')
    #plt.title('Mass Flow Rate vs Run Time')
    #plt.legend()
    #plt.ylim(bottom=0)
    #plt.tight_layout()

    lns1 = ax1.plot(x, df['Differential Pressure [Bar]'],label='Differential Pressure')
    ax1.set_xlabel('Run Time [s]', fontdict=font_axis)
    ax1.set_ylabel('Differential Pressure [Bar]', fontdict=font_axis)
    ax1.set_ylim(bottom=0)
    
    lns2 = ax2.plot(x, df["Torque [N-m]"],color="orange",label='Torque')
    ax2.set_ylabel('Torque [N-m]',fontsize = 16, fontdict=font_axis)
    ax2.set_ylim([0,0.6])
    
    lns = lns1 + lns2

    labs = [l.get_label() for l in lns]       
    ax1.legend(lns, labs, loc="lower right",prop=font_legend)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_Pressure_&_Torque_vs_Run_Time_full_combined_plot.png"))
    plt.close()

def main(csv_dir):
    files = glob.glob(os.path.join(csv_dir, '*.csv'))
    if not files:
        print(f"No CSV files found in {csv_dir}")
        return

    summaries = [process_file(f) for f in files]

    
    
if __name__ == '__main__':
    # 1) Base path (up to “Glycerine\”)
    BASE_DIR = r'C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine'
    
    # 2) List the final two-level folders you want to process:
    subdirs = [
        r'24042025\Glycerine_8%_6730_0.081Kg-s',
        r'24042025\Glycerine_8%_7760_0.094Kg-s',
        r'23042025\Glycerine_8%_7250_0.094Kg-s',
        r'23042025\Glycerine_85%_6730_0.526Kg-s',
        r'23042025\Glycerine_85%_7760_0.603Kg-s',
        r'22042025\Glycerine_52.5%_6730_0.454Kg-s',
        r'22042025\Glycerine_52.5%_7760_0.521Kg-s',
        r'22042025\Glycerine_85%_7250_0.565Kg-s',
        r'21042025\Glycerine_43%_6730_0.416Kg-s',
        r'21042025\Glycerine_43%_7250_0.447Kg-s',
        r'21042025\Glycerine_43%_7760_0.477Kg-s',
        r'21042025\Glycerine_52.5%_7250_0.487Kg-s',
        r'20042025\Glycerine_31.5%_6730_0.344Kg-s',
        r'20042025\Glycerine_31.5%_7250_0.370Kg-s',
        r'20042025\Glycerine_36.5%_6730_0.378Kg-s',
        r'20042025\Glycerine_36.5%_7250_0.408Kg-s',
        r'19042025\Glycerine_13%_6730_0.160Kg-s',
        r'19042025\Glycerine_13%_7760_0.188Kg-s',
        r'18042025\Glycerine_13%_7250_0.175Kg-s',
        r'18042025\Glycerine_19%_7760_0.268Kg-s',
        r'17042025\Glycerine_19%_6730_0.235Kg-s',
        r'17042025\Glycerine_19%_7250_0.253Kg-s',
        r'17042025\Glycerine_25.6%_6730_0.302Kg-s',
        r'17042025\Glycerine_25.6%_7250_0.328Kg-s',
        r'16042025\Glycerine_21%_6730_0.272Kg-s',
        r'26042025\Glycerine_27.5%_7760_0.354Kg-s',
        r'26042025\Glycerine_21%_7760_0.313Kg-s',
        r'26042025\Glycerine_21%_7250_0.281Kg-s',
        r'28042025\Glycerine_13%_6730_0.160Kg-s',
        r'28042025\Glycerine_21%_7250_0.290Kg-s',
        r'28042025\Glycerine_31.5%_7760_0.395Kg-s',
        r'28042025\Glycerine_36.5_7760_0.437Kg-s',
        r'28042025\Glycerine_52.5%_7760_0.521Kg-s',
        r'29042025\Glycerine_13%_7760_0.187Kg-s',
    ]

    # 3) Iterate through each and run
    for sub in subdirs:
        csv_directory = os.path.join(BASE_DIR, sub)
        print(f"\n=== Processing folder: {csv_directory} ===")
        main(csv_directory)



=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\24042025\Glycerine_8%_6730_0.081Kg-s ===

=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\24042025\Glycerine_8%_7760_0.094Kg-s ===

=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\23042025\Glycerine_8%_7250_0.094Kg-s ===

=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\23042025\Glycerine_85%_6730_0.526Kg-s ===

=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\23042025\Glycerine_85%_7760_0.603Kg-s ===

=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glyce

In [54]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.stats import t

def process_file(file_path):
    font_legend = {'family': 'TH Sarabun New',
                   'weight': 'bold',
                   'style': 'normal',
                   'size': 18}
    font_axis = {'family': 'TH Sarabun New',
                 'weight': 'bold',
                 'style': 'normal',
                 'size': 20}

    base_name  = os.path.splitext(os.path.basename(file_path))[0]
    parent_dir = os.path.dirname(file_path)
    output_dir = os.path.join(parent_dir, f"{base_name} plots")
    os.makedirs(output_dir, exist_ok=True)

    # Read CSV, skip first two summary rows, use third row as header
    df = pd.read_csv(file_path, delimiter=',', skiprows=2, header=0)
    x  = df['Run Time [s]']

    # 1) Mass Flow Rate vs Run Time (full) — raw data only, no curve fit
    plt.figure()
    plt.plot(x, df['Weight [Kg]'], label='Data')
    plt.xlabel('Run Time [s]', fontdict=font_axis)
    plt.ylabel('Weight [Kg]', fontdict=font_axis)
    plt.title('Mass Flow Rate vs Run Time', fontdict=font_axis)
    plt.legend(prop=font_legend)
    plt.ylim(bottom=0)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_name}_Mass_Flow_Rate_full_plot.png"))
    plt.close()

    # ... (Differential Pressure, Rotational Speed, Torque full-plot blocks unchanged) ...

    # --- FILTERED PLOTS (20 ≤ Weight ≤ 30 Kg) ---
    filtered = df[(df['Weight [Kg]'] >= 20) & (df['Weight [Kg]'] <= 30)]
    if not filtered.empty:
        
        frt = filtered['Run Time [s]'].iloc[-1] - filtered['Run Time [s]'].iloc[0]

        # Mass Flow Rate (filtered fit) + capture its standard error
        xf, yf = filtered['Run Time [s]'], filtered['Weight [Kg]']
        slope_f, intercept_f, r_f, _, stderr_f = linregress(xf, yf)
        # convert 1σ stderr into 95% CI half-width
        dof    = len(xf) - 2
        t_crit = t.ppf(0.975, dof)
        stderr_f *= t_crit
        r2_f = r_f**2
        max_mass = max(yf)
        min_mass = min(yf)
        plt.figure()
        plt.plot(xf, yf, label='Filtered')
        plt.plot(xf, slope_f*xf + intercept_f,
                 label=f'Fit: y={slope_f:.4f}x+{intercept_f:.4f}\nR²={r2_f:.4f}')
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Weight [Kg]', fontdict=font_axis)
        plt.title('Mass Flow Rate vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95, f'Filtered Run Time: {frt:.2f} s',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.legend(prop=font_legend); plt.ylim(bottom=0)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Mass_Flow_Rate_filtered_plot.png"))
        plt.close()

        # Differential Pressure stats + plot
        dp     = filtered['Differential Pressure [Bar]']
        avg_dp = dp.mean()
        std_dp = 2 * dp.std(ddof=1)
        plt.figure()
        plt.plot(xf, dp)
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Differential Pressure [Bar]', fontdict=font_axis)
        plt.title('Differential Pressure vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s\n'
                 f'Avg: {avg_dp:.4f} Bar\n'
                 f'SD:  {std_dp:.4f}',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.ylim([0,3])
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Differential_Pressure_filtered_plot.png"))
        plt.close()

        # Rotational Speed stats + plot
        rs     = filtered['Rotational Speed [RPM]']
        avg_rs = rs.mean()
        std_rs = 2 * rs.std(ddof=1)
        plt.figure()
        plt.plot(xf, rs)
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Rotational Speed [RPM]', fontdict=font_axis)
        plt.title('Rotational Speed vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s\n'
                 f'Avg: {avg_rs:.4f} RPM\n'
                 f'SD:  {std_rs:.4f}',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.ylim([0,8500])
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Rotational_Speed_filtered_plot.png"))
        plt.close()

        # Torque stats + plot
        tq     = filtered['Torque [N-m]']
        avg_tq = tq.mean()
        std_tq = 2 * tq.std(ddof=1)
        plt.figure()
        plt.plot(xf, tq)
        plt.xlabel('Run Time [s]', fontdict=font_axis)
        plt.ylabel('Torque [N-m]', fontdict=font_axis)
        plt.title('Torque vs Run Time (Filtered)', fontdict=font_axis)
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s\n'
                 f'Avg: {avg_tq:.4f} N-m\n'
                 f'SD:  {std_tq:.4f}',
                 transform=plt.gca().transAxes, verticalalignment='top', fontdict=font_axis)
        plt.ylim([0,0.5])
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{base_name}_Torque_filtered_plot.png"))
        plt.close()
        
    else:
        slope_f = stderr_f = np.nan
        avg_dp = avg_rs = avg_tq = std_dp = std_rs = std_tq = np.nan
        print(f"{base_name}: no data in weight range 20–30 Kg")

    # ---- RETURN original metrics PLUS S.E. of slope ----
    return {
        'file':                base_name,
        'mass_flow_rate':      slope_f,
        'se_mass_flow_rate':   stderr_f,     # added standard error
        'mean_dp':             avg_dp,
        'mean_rs':             avg_rs,
        'mean_torque':         avg_tq,
        'std_dp':              std_dp,
        'std_rs':              std_rs,
        'std_tq':              std_tq,
        'run_time':            frt,
        'max_mass':            max_mass,
        'min_mass':            min_mass
    }


def main(csv_dir):
    files = glob.glob(os.path.join(csv_dir, '*.csv'))
    if not files:
        print(f"No CSV files found in {csv_dir}")
        return

    summaries = [process_file(f) for f in files]

    # ARRAYS of the PER-FILE time-averaged MEANS (unchanged)
    mfr_arr = np.array([s['mass_flow_rate'] for s in summaries])
    dp_arr  = np.array([s['mean_dp']        for s in summaries])
    rs_arr  = np.array([s['mean_rs']        for s in summaries])
    tq_arr  = np.array([s['mean_torque']    for s in summaries])
    run_time_arr = np.array([s['run_time']    for s in summaries])
    max_mass_arr = np.array([s['max_mass']    for s in summaries])
    min_mass_arr = np.array([s['min_mass']    for s in summaries])
    
    # ARRAYS of the PER-FILE SDs (unchanged)
    sd_dp_arr = np.array([s['std_dp'] for s in summaries])
    sd_rs_arr = np.array([s['std_rs'] for s in summaries])
    sd_tq_arr = np.array([s['std_tq'] for s in summaries])

    # ARRAY of the PER-FILE S.E.
    se_arr = np.array([s['se_mass_flow_rate'] for s in summaries])

    # GRAND MEANS & SDs of the MEANS (unchanged)
    grand_means = {
        'mfr_mean':  np.nanmean(mfr_arr), 'mfr_sd':  np.nanstd(mfr_arr, ddof=1),
        'dp_mean':   np.nanmean(dp_arr),  'dp_sd':   np.nanstd(dp_arr,  ddof=1),
        'rs_mean':   np.nanmean(rs_arr),  'rs_sd':   np.nanstd(rs_arr,  ddof=1),
        'tq_mean':   np.nanmean(tq_arr),  'tq_sd':   np.nanstd(tq_arr,  ddof=1),
        'run_time_mean':   np.nanmean(run_time_arr),  'run_time_sd':   np.nanstd(run_time_arr,  ddof=1),
        'max_mass_mean':   np.nanmean(max_mass_arr),  'max_mass_sd':   np.nanstd(max_mass_arr,  ddof=1),
        'min_mass_mean':   np.nanmean(min_mass_arr),  'min_mass_sd':   np.nanstd(min_mass_arr,  ddof=1),
    } #Single Number

    # ARITHMETIC AVERAGE OF THE SDs (unchanged)
    avg_sd = {
        'dp_bar': np.nanmean(sd_dp_arr),
        'rs_bar': np.nanmean(sd_rs_arr),
        'tq_bar': np.nanmean(sd_tq_arr),
    }

    # AVERAGE STANDARD ERROR OF MASS FLOW RATE (added)
    avg_se = np.nanmean(se_arr)

    #Precision Uncertainty Calculation Initialization
    Ns = len(dp_arr)
    dof = Ns - 1  # degrees of freedom for mean
    t_crit = t.ppf(0.975, dof) #0.975 for two-tails student-t
    
    grand_pu = {
        'mfr_pu':  t_crit * np.nanstd(mfr_arr, ddof=1) / np.sqrt(Ns),
        'dp_pu':   t_crit * np.nanstd(dp_arr, ddof=1) / np.sqrt(Ns),
        'rs_pu':   t_crit * np.nanstd(rs_arr, ddof=1) / np.sqrt(Ns),
        'tq_pu':   t_crit * np.nanstd(tq_arr, ddof=1) / np.sqrt(Ns)
    } #Single Number
    
    #Bias Uncertainty Code Area
    Accuracy_Dict = {
        'mass_acc': 0.01, 'mass_res': 0.01, #kg
        'time_acc': 50e-6 * 0.05, 'time_res': 50e-9, #sec
        'dp_acc': 0.0075, 'dp_res': 10 / 2**16 * 0.3, #bar, res = 10V / 16-bit * 0.3 Bar/V, acc = 0.25%FS
        'tq_acc': 0.01, 'tq_res': 0.01, #N-m
        'rs_acc': 0, 'rs_res': 0 #RPM   
    } #Single Number
    
    mass_bu = np.sqrt((Accuracy_Dict['mass_acc'])**2 + (Accuracy_Dict['mass_res'] / 2)**2)
    time_bu = np.sqrt((Accuracy_Dict['time_acc'])**2 + (Accuracy_Dict['time_res'] / 2)**2)
    dp_bu = np.sqrt((Accuracy_Dict['dp_acc'])**2 + (Accuracy_Dict['dp_res'] / 2)**2)
    rs_bu = np.sqrt((Accuracy_Dict['rs_acc'])**2 + (Accuracy_Dict['rs_res'] / 2)**2)
    tq_bu = np.sqrt((Accuracy_Dict['tq_acc'])**2 + (Accuracy_Dict['tq_res'] / 2)**2)
    mfr_bu = np.sqrt((mass_bu/grand_means['run_time_mean'])**2 + ((grand_means['max_mass_mean'] - grand_means['min_mass_mean'])/grand_means['run_time_mean']**2*time_bu)**2)
    
    #Final Sum Uncertainty
    mfr_uncertainty = np.sqrt((mfr_bu)**2 + (grand_pu['mfr_pu'])**2 + (avg_se)**2)
    dp_uncertainty = np.sqrt((dp_bu)**2 + (dp_bu)**2 + (avg_sd['dp_bar'])**2)
    rs_uncertainty = np.sqrt((rs_bu)**2 + (rs_bu)**2 + (avg_sd['rs_bar'])**2)
    tq_uncertainty = np.sqrt((tq_bu)**2 + (tq_bu)**2 + (avg_sd['tq_bar'])**2)
    
    # --- PRINT ---
    print("\nPer-file filtered MEANS:")
    for s in summaries:
        print(f" - {s['file']}: Mass Flow Rate={s['mass_flow_rate']:.4f}, "
              f"ΔPs={s['mean_dp']:.4f}, RPM={s['mean_rs']:.2f}, "
              f"Torque={s['mean_torque']:.4f}")

    print("\nGrand trimmed summary (of MEANS):")
    print(f" • Mass Flow Rate: mean={grand_means['mfr_mean']:.4f}, SD={grand_means['mfr_sd']:.4f}, Precision Uncertainty={grand_pu['mfr_pu']:.4f}, Bias Uncertainty={mfr_bu:.4f}")
    print(f" • ΔPs:             mean={grand_means['dp_mean']:.4f}, SD={grand_means['dp_sd']:.4f}, Precision Uncertainty={grand_pu['dp_pu']:.4f}, Bias Uncertainty={dp_bu:.4f}")
    print(f" • RPM:            mean={grand_means['rs_mean']:.2f}, SD={grand_means['rs_sd']:.2f}, Precision Uncertainty={grand_pu['rs_pu']:.4f}, Bias Uncertainty={rs_bu:.4f}")
    print(f" • Torque:         mean={grand_means['tq_mean']:.4f}, SD={grand_means['tq_sd']:.4f}, Precision Uncertainty={grand_pu['tq_pu']:.4f}, Bias Uncertainty={tq_bu:.4f}")


    print("\nPer-file filtered SDs:")
    for s in summaries:
        print(
            f" - {s['file']}: "
            f"SE Mass Flow Rate={s['se_mass_flow_rate']:.4f}, "
            f"2*SD ΔPs={s['std_dp']:.4f}, "
            f"2*SD RPM={s['std_rs']:.4f}, "
            f"2*SD Torque={s['std_tq']:.4f}"
        )

    print("\nAverage of SDs (SD̄):")
    print(f" • ΔPs 2*SD̄ = {avg_sd['dp_bar']:.4f}")
    print(f" • RPM 2*SD̄= {avg_sd['rs_bar']:.4f}")
    print(f" • Torque 2*SD̄ = {avg_sd['tq_bar']:.4f}")

    # Print average standard error
    print(f"\nAverage standard error of Mass Flow Rate = {avg_se:.4f}\n")

    # --- WRITE summary.txt ---
    folder = os.path.basename(csv_dir.rstrip(os.sep))
    summary_path = os.path.join(csv_dir, f"{folder} summary.txt")
    with open(summary_path, 'w', encoding='utf-8') as f:
        
        f.write("\nTime-Averaged Values:\n")
        for s in summaries:
            f.write(f"{s['file']}: Mass Flow Rate={s['mass_flow_rate']:.4f}, "
                    f"ΔPs={s['mean_dp']:.4f}, RPM={s['mean_rs']:.2f}, "
                    f"Torque={s['mean_torque']:.4f}\n")
            
        f.write("\nTime-Averaged S.D. / S.E.:\n")
        
        for s in summaries:
            f.write(
                f"{s['file']}: "
                f"SE (95% CI) Mass Flow Rate={s['se_mass_flow_rate']:.4f}, "
                f"2*SD ΔPs={s['std_dp']:.4f}, "
                f"2*SD RPM={s['std_rs']:.4f}, "
                f"2*SD Torque={s['std_tq']:.4f}\n"
            )
        
        f.write("\nInstance Average of Means Value:\n")
        f.write(f"Mass Flow Rate: mean={grand_means['mfr_mean']:.4f}, SD={grand_means['mfr_sd']:.4f}, Precision Uncertainty={grand_pu['mfr_pu']:.4f}, Bias Uncertainty={mfr_bu:.4f}\n")
        f.write(f"ΔPs:             mean={grand_means['dp_mean']:.4f}, SD={grand_means['dp_sd']:.4f}, Precision Uncertainty={grand_pu['dp_pu']:.4f}, Bias Uncertainty={dp_bu:.4f}\n")
        f.write(f"RPM:            mean={grand_means['rs_mean']:.2f}, SD={grand_means['rs_sd']:.2f}, Precision Uncertainty={grand_pu['rs_pu']:.4f}, Bias Uncertainty={rs_bu:.4f}\n")
        f.write(f"Torque:         mean={grand_means['tq_mean']:.4f}, SD={grand_means['tq_sd']:.4f}, Precision Uncertainty={grand_pu['tq_pu']:.4f}, Bias Uncertainty={tq_bu:.4f}\n")
        
        f.write("\nInstance Average of Time-Averaged SDs (SD̄):\n")
        f.write(f"S.E. (95% CI) of Mass Flow Rate = {avg_se:.4f}\n")
        f.write(f"ΔPs 2*SD̄ = {avg_sd['dp_bar']:.4f}\n")
        f.write(f"RPM 2*SD̄= {avg_sd['rs_bar']:.4f}\n")
        f.write(f"Torque 2*SD̄ = {avg_sd['tq_bar']:.4f}\n")

        f.write("\nFinal Uncertainty:\n")
        f.write(f"Uncertainty of Mass Flow Rate = {mfr_uncertainty:.4f} Kg/s\n")
        f.write(f"Uncertainty of ΔPs (RAW) = {dp_uncertainty:.4f} Bar\n")
        f.write(f"Uncertainty of RPM = {rs_uncertainty:.4f} RPM\n")
        f.write(f"Uncertainty of Torque = {tq_uncertainty:.4f} N-m\n")


    print(f"Summary written to {summary_path}")

if __name__ == '__main__':
    # 1) Base path (up to “Glycerine\”)
    BASE_DIR = r'C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine'
    
    # 2) List the final two-level folders you want to process:
    subdirs = [
        r'29042025\Glycerine_13%_7760_0.187Kg-s',
    ]

    # 3) Iterate through each and run
    for sub in subdirs:
        csv_directory = os.path.join(BASE_DIR, sub)
        print(f"\n=== Processing folder: {csv_directory} ===")
        main(csv_directory)



=== Processing folder: C:\Users\Krai\OneDrive - Chulalongkorn University\Electric Pump Senior Project\Experimental Record\Glycerine\29042025\Glycerine_13%_7760_0.187Kg-s ===

Per-file filtered MEANS:
 - Glycerine_29042025_1221_7760RPM_13%_#3_0.184Kg-s: Mass Flow Rate=0.1832, ΔPs=2.6313, RPM=7759.96, Torque=0.3912
 - Glycerine_29042025_1234_7760RPM_13%_#4_0.184Kg-s: Mass Flow Rate=0.1836, ΔPs=2.6318, RPM=7759.87, Torque=0.3891
 - Glycerine_29042025_1732_7760RPM_13%_#6_0.184Kg-s: Mass Flow Rate=0.1831, ΔPs=2.6193, RPM=7759.92, Torque=0.3963
 - Glycerine_29042025_1756_7760RPM_13%_#7_0.184Kg-s: Mass Flow Rate=0.1836, ΔPs=2.6291, RPM=7759.84, Torque=0.4022
 - Glycerine_29042025_1949_7760RPM_13%_#11_0.183Kg-s: Mass Flow Rate=0.1815, ΔPs=2.6264, RPM=7759.80, Torque=0.3987
 - Glycerine_29042025_2034_7760RPM_13%_#13_0.181Kg-s: Mass Flow Rate=0.1807, ΔPs=2.6362, RPM=7759.84, Torque=0.3842

Grand trimmed summary (of MEANS):
 • Mass Flow Rate: mean=0.1826, SD=0.0012, Precision Uncertainty=0.0013,